In [18]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [26]:
import tiktoken
import pandas as pd
from ast import literal_eval
import gpt_pairwise
from urllib import parse
from tqdm import tqdm
import random

In [20]:
def fix_title(title):
    return parse.unquote(title).replace('_', ' ')


In [21]:
df = pd.read_parquet('/dlabdata1/tsoares/linkrec-llms/data_modelling/benchmark_multilingual/test_data/en.parquet')
df.head(5)

,source_title,source_ID,target_title,context,section,mention_present,source_lead,first_version,second_version,direct_match,...,roberta_simple_multilingual,roberta_full_en,roberta_full_multilingual,roberta_dyn_mask_en,roberta_only_expansion_en,roberta_expansion_en,roberta_full_multilingual-all,entqa_rank,get_rank,roberta_dyn_mask_no_neg_en
0,2023%E2%80%9324_AFC_Cup_group_stage,74555776,Hettikamkanamge_Perera,Group A\nSource: Rules for classification: Gro...,Ranking of runner-up teams,False,AFC Cup group stage\nThe 2023–24 AFC Cup group...,1178732500,1178733621,True,...,1,1,1,3,1,1,1,3,2,1
1,Madonna_and_business,75009374,MCI_Group,"From 2017 to 2020, Madonna lived in Lisbon, Po...",Commercial impact,False,Aspect of Madonna's career\nAmerican singer-so...,1179712520,1179715108,False,...,119,21,1,66,30,76,55,107,17,14
2,2024_UEFA_Women%27s_Under-19_Championship_qual...,74509532,Bulgarian_Football_Union,Source: UEFARules for classification: Qualific...,Round 1,False,International football competition\nThe 2024 U...,1181645363,1181645710,False,...,12,11,2,39,1,6,1,41,32,19
3,Racism_in_Quebec,75111544,Bouchard-Taylor_Commission,"Historically, there has been a reluctance in Q...",History,True,Prejudice towards immigrants and minorities fr...,1181203098,1181204380,True,...,1,18,1,1,2,3,1,149,1,1
4,India%27s_Got_Talent_%28season_10%29,71081798,Ravi_Teja,The team with the lowest score goes to the eli...,Elimination Chart,False,Indian TV show\nThe tenth season of Indian tal...,1180115976,1180221202,True,...,1,5,3,1,3,3,4,10,11,1


# Prompt strategy experiments

In [22]:
df_test = df[df['missing_category'].isna()].sample(5)
df_test

,source_title,source_ID,target_title,context,section,mention_present,source_lead,first_version,second_version,direct_match,...,roberta_simple_multilingual,roberta_full_en,roberta_full_multilingual,roberta_dyn_mask_en,roberta_only_expansion_en,roberta_expansion_en,roberta_full_multilingual-all,entqa_rank,get_rank,roberta_dyn_mask_no_neg_en
503,1990_Santos_FC_season,74845591,Santos_FC,Futures v Santos\nCompetitions Campeonato Paul...,Competitions,True,Santos 1990 football season\nThe 1990 season w...,1177160232,1177172712,True,...,1,1,1,1,1,1,1,1,1,1
704,St._Mary%27s_School_%28Knoxville%2C_Illinois%29,74941861,Carrara_marble,Many of the rooms were elaborately furnished. ...,Architecture and fittings,True,St. Mary's School was an American parochial gi...,1177845080,1177845855,True,...,1,6,4,1,3,1,3,6,1,1
213,2023_Ryder_Cup,58628035,Justin_Thomas,Sunday's matches\nThe first match between Jon ...,Event summary,True,Men's golf competition between the United Stat...,1178705998,1178710751,True,...,1,1,1,1,1,1,1,30,1,1
460,2023%E2%80%9324_BG_Pathum_United_F.C._season,73930601,Lamphun_Warriors_F.C.,Appearances and goals\nAs of 7 Oct 2023\nCompe...,Competitions,True,Association football season\nBG Pathum United ...,1180432149,1180436472,True,...,1,1,1,1,1,1,1,3,1,1
652,Caitlin_Clark,67294481,DePaul_Blue_Demons_women%27s_basketball,She was a unanimous first-team All-American fo...,College career,True,American basketball player (born 2002)\nThis a...,1180331455,1180331635,True,...,1,2,1,1,1,2,1,7,10,2


In [23]:
# prepare the inputs
target_titles = df_test['target_title'].apply(fix_title).tolist()
target_leads = df_test['target_lead'].to_list()

target_pages = [{'target_title': title,
'target_lead': lead} for title, lead in zip(target_titles, target_leads)]

df_rows = df_test.to_dict('records')
all_contexts = []
counter = 0
for row in tqdm(df_rows):
    positive_context = {'section_title': row['section'],
                        'link_context': row['context']}
    negative_contexts = literal_eval(row['negative_contexts'])
    row_contexts = []
    for context in negative_contexts:
        negative_context = {'section_title': context['section'],
                            'link_context': context['context']}
        row_contexts.append([[positive_context, negative_context], [negative_context, positive_context]])
        counter += 2
    all_contexts.append(row_contexts)

print(f'Generated {counter} contexts for language')

100%|██████████| 5/5 [00:00<00:00, 192.82it/s]

Generated 480 contexts for language


# PROMPT EXPERIMENTS

In [24]:
gpt_results = {}

## Prompt Strategy 1

In [25]:
gpt_results['1'] = [] 
for (target_page, contexts) in (pbar := tqdm(zip(target_pages, all_contexts), total=len(target_pages))):
    gpt_results['1'].append([])
    for i, context_pair in enumerate(contexts):
        pbar.set_description(f'{i} / {len(contexts)}')
        answers = []
        for context in context_pair:
            result = gpt_pairwise.get_best_candidate(context, target_page, 'gpt-3.5-turbo-0125', prompt_type=1)
            if result == None:
                print('ERROR')
                answers.append(None)
            else:
                answers.append(result.choices[0].message.content)
        gpt_results['1'][-1].append(answers)

0 / 2:   0%|          | 0/5 [00:00<?, ?it/s]

107 / 108: 100%|██████████| 5/5 [08:32<00:00, 102.40s/it]


In [27]:
positions = []
successes = 0
ties = 0
fails = 0
for results in gpt_results['1']:
    position = 1
    tied = 0
    # results = results.replace('{{', '').replace('}}', '')
    # results = literal_eval(results)
    for result in results:
        simple_result = []
        if 'Passage A' in result[0] and 'Passage B' not in result[0]:
            simple_result.append('A')
        elif 'Passage B' in result[0] and 'Passage A' not in result[0]:
            simple_result.append('B')
        else:
            simple_result.append(result[0])
        
        if 'Passage A' in result[1] and 'Passage B' not in result[1]:
            simple_result.append('A')
        elif 'Passage B' in result[1] and 'Passage A' not in result[1]:
            simple_result.append('B')
        else:
            simple_result.append(result[1])
        
        
        if simple_result == ['B', 'B'] or simple_result == ['A', 'A']:
            tied += 1
            ties += 1
        elif simple_result == ['A', 'B']:
            successes += 1
            continue
        elif simple_result == ['B', 'A']:
            position += 1
            fails += 1
        else:
            print(result)
    positions.append(position + random.randint(0, tied))
print(successes, ties, fails)
print(positions)


203 36 1
[1, 1, 23, 1, 2]


# Prompt Strategy 2

In [28]:
gpt_results['2'] = [] 
for (target_page, contexts) in (pbar := tqdm(zip(target_pages, all_contexts), total=len(target_pages))):
    gpt_results['2'].append([])
    for i, context_pair in enumerate(contexts):
        pbar.set_description(f'{i} / {len(contexts)}')
        answers = []
        for context in context_pair:
            result = gpt_pairwise.get_best_candidate(context, target_page, 'gpt-3.5-turbo-0125', prompt_type=1)
            if result == None:
                print('ERROR')
                answers.append(None)
            else:
                answers.append(result.choices[0].message.content)
        gpt_results['2'][-1].append(answers)

107 / 108: 100%|██████████| 5/5 [08:13<00:00, 98.77s/it] 


In [29]:
positions = []
successes = 0
ties = 0
fails = 0
for results in gpt_results['2']:
    position = 1
    tied = 0
    # results = results.replace('{{', '').replace('}}', '')
    # results = literal_eval(results)
    for result in results:
        simple_result = []
        if 'Passage A' in result[0] and 'Passage B' not in result[0]:
            simple_result.append('A')
        elif 'Passage B' in result[0] and 'Passage A' not in result[0]:
            simple_result.append('B')
        else:
            simple_result.append(result[0])
        
        if 'Passage A' in result[1] and 'Passage B' not in result[1]:
            simple_result.append('A')
        elif 'Passage B' in result[1] and 'Passage A' not in result[1]:
            simple_result.append('B')
        else:
            simple_result.append(result[1])
        
        
        if simple_result == ['B', 'B'] or simple_result == ['A', 'A']:
            tied += 1
            ties += 1
        elif simple_result == ['A', 'B']:
            successes += 1
            continue
        elif simple_result == ['B', 'A']:
            position += 1
            fails += 1
        else:
            print(result)
    positions.append(position + random.randint(0, tied))
print(successes, ties, fails)
print(positions)


200 40 0
[2, 1, 11, 1, 10]


# Prompt Strategy 3

In [30]:
gpt_results['3'] = [] 
for (target_page, contexts) in (pbar := tqdm(zip(target_pages, all_contexts), total=len(target_pages))):
    gpt_results['3'].append([])
    for i, context_pair in enumerate(contexts):
        pbar.set_description(f'{i} / {len(contexts)}')
        answers = []
        for context in context_pair:
            result = gpt_pairwise.get_best_candidate(context, target_page, 'gpt-3.5-turbo-0125', prompt_type=1)
            if result == None:
                print('ERROR')
                answers.append(None)
            else:
                answers.append(result.choices[0].message.content)
        gpt_results['3'][-1].append(answers)

107 / 108: 100%|██████████| 5/5 [08:26<00:00, 101.27s/it]


In [31]:
positions = []
successes = 0
ties = 0
fails = 0
for results in gpt_results['3']:
    position = 1
    tied = 0
    # results = results.replace('{{', '').replace('}}', '')
    # results = literal_eval(results)
    for result in results:
        simple_result = []
        if 'Passage A' in result[0] and 'Passage B' not in result[0]:
            simple_result.append('A')
        elif 'Passage B' in result[0] and 'Passage A' not in result[0]:
            simple_result.append('B')
        else:
            simple_result.append(result[0])
        
        if 'Passage A' in result[1] and 'Passage B' not in result[1]:
            simple_result.append('A')
        elif 'Passage B' in result[1] and 'Passage A' not in result[1]:
            simple_result.append('B')
        else:
            simple_result.append(result[1])
        
        
        if simple_result == ['B', 'B'] or simple_result == ['A', 'A']:
            tied += 1
            ties += 1
        elif simple_result == ['A', 'B']:
            successes += 1
            continue
        elif simple_result == ['B', 'A']:
            position += 1
            fails += 1
        else:
            print(result)
    positions.append(position + random.randint(0, tied))
print(successes, ties, fails)
print(positions)

205 34 1
[2, 1, 24, 1, 1]


# Prompt Strategy 4

In [32]:
gpt_results['4'] = [] 
for (target_page, contexts) in (pbar := tqdm(zip(target_pages, all_contexts), total=len(target_pages))):
    gpt_results['4'].append([])
    for i, context_pair in enumerate(contexts):
        pbar.set_description(f'{i} / {len(contexts)}')
        answers = []
        for context in context_pair:
            result = gpt_pairwise.get_best_candidate(context, target_page, 'gpt-3.5-turbo-0125', prompt_type=1)
            if result == None:
                print('ERROR')
                answers.append(None)
            else:
                answers.append(result.choices[0].message.content)
        gpt_results['4'][-1].append(answers)

107 / 108: 100%|██████████| 5/5 [07:55<00:00, 95.05s/it] 


In [33]:
positions = []
successes = 0
ties = 0
fails = 0
for results in gpt_results['4']:
    position = 1
    tied = 0
    # results = results.replace('{{', '').replace('}}', '')
    # results = literal_eval(results)
    for result in results:
        simple_result = []
        if 'Passage A' in result[0] and 'Passage B' not in result[0]:
            simple_result.append('A')
        elif 'Passage B' in result[0] and 'Passage A' not in result[0]:
            simple_result.append('B')
        else:
            simple_result.append(result[0])
        
        if 'Passage A' in result[1] and 'Passage B' not in result[1]:
            simple_result.append('A')
        elif 'Passage B' in result[1] and 'Passage A' not in result[1]:
            simple_result.append('B')
        else:
            simple_result.append(result[1])
        
        
        if simple_result == ['B', 'B'] or simple_result == ['A', 'A']:
            tied += 1
            ties += 1
        elif simple_result == ['A', 'B']:
            successes += 1
            continue
        elif simple_result == ['B', 'A']:
            position += 1
            fails += 1
        else:
            print(result)
    positions.append(position + random.randint(0, tied))
print(successes, ties, fails)
print(positions)

205 34 1
[2, 1, 14, 1, 1]


# Prompt Strategy 5

In [34]:
gpt_results['5'] = [] 
for (target_page, contexts) in (pbar := tqdm(zip(target_pages, all_contexts), total=len(target_pages))):
    gpt_results['5'].append([])
    for i, context_pair in enumerate(contexts):
        pbar.set_description(f'{i} / {len(contexts)}')
        answers = []
        for context in context_pair:
            result = gpt_pairwise.get_best_candidate(context, target_page, 'gpt-3.5-turbo-0125', prompt_type=1)
            if result == None:
                print('ERROR')
                answers.append(None)
            else:
                answers.append(result.choices[0].message.content)
        gpt_results['5'][-1].append(answers)

107 / 108: 100%|██████████| 5/5 [06:39<00:00, 79.97s/it]


In [35]:
positions = []
successes = 0
ties = 0
fails = 0
for results in gpt_results['5']:
    position = 1
    tied = 0
    # results = results.replace('{{', '').replace('}}', '')
    # results = literal_eval(results)
    for result in results:
        simple_result = []
        if 'Passage A' in result[0] and 'Passage B' not in result[0]:
            simple_result.append('A')
        elif 'Passage B' in result[0] and 'Passage A' not in result[0]:
            simple_result.append('B')
        else:
            simple_result.append(result[0])
        
        if 'Passage A' in result[1] and 'Passage B' not in result[1]:
            simple_result.append('A')
        elif 'Passage B' in result[1] and 'Passage A' not in result[1]:
            simple_result.append('B')
        else:
            simple_result.append(result[1])
        
        
        if simple_result == ['B', 'B'] or simple_result == ['A', 'A']:
            tied += 1
            ties += 1
        elif simple_result == ['A', 'B']:
            successes += 1
            continue
        elif simple_result == ['B', 'A']:
            position += 1
            fails += 1
        else:
            print(result)
    positions.append(position + random.randint(0, tied))
print(successes, ties, fails)
print(positions)

201 38 1
[2, 1, 14, 1, 1]


# Strategy 2 with GPT-4

In [36]:
gpt_results['2-GPT4'] = [] 
for (target_page, contexts) in (pbar := tqdm(zip(target_pages, all_contexts), total=len(target_pages))):
    gpt_results['2-GPT4'].append([])
    for i, context_pair in enumerate(contexts):
        pbar.set_description(f'{i} / {len(contexts)}')
        answers = []
        for context in context_pair:
            result = gpt_pairwise.get_best_candidate(context, target_page, 'gpt-4-0125-preview', prompt_type=1)
            if result == None:
                print('ERROR')
                answers.append(None)
            else:
                answers.append(result.choices[0].message.content)
        gpt_results['2-GPT4'][-1].append(answers)

25 / 46:  20%|██        | 1/5 [03:53<01:34, 23.61s/it]

ERROR


32 / 46:  20%|██        | 1/5 [06:27<01:34, 23.61s/it]

ERROR


1 / 81:  40%|████      | 2/5 [11:36<18:52, 377.52s/it] 

ERROR


107 / 108: 100%|██████████| 5/5 [23:11<00:00, 278.37s/it]


In [38]:
positions = []
successes = 0
ties = 0
fails = 0
for results in gpt_results['2-GPT4']:
    position = 1
    tied = 0
    # results = results.replace('{{', '').replace('}}', '')
    # results = literal_eval(results)
    for result in results:
        simple_result = []
        if result[0] is None or result[1] is None:
            continue
        if 'Passage A' in result[0] and 'Passage B' not in result[0]:
            simple_result.append('A')
        elif 'Passage B' in result[0] and 'Passage A' not in result[0]:
            simple_result.append('B')
        else:
            simple_result.append(result[0])
        
        if 'Passage A' in result[1] and 'Passage B' not in result[1]:
            simple_result.append('A')
        elif 'Passage B' in result[1] and 'Passage A' not in result[1]:
            simple_result.append('B')
        else:
            simple_result.append(result[1])
        
        
        if simple_result == ['B', 'B'] or simple_result == ['A', 'A']:
            tied += 1
            ties += 1
        elif simple_result == ['A', 'B']:
            successes += 1
            continue
        elif simple_result == ['B', 'A']:
            position += 1
            fails += 1
        else:
            print(result)
    positions.append(position + random.randint(0, tied))
print(successes, ties, fails)
print(positions)

237 0 0
[1, 1, 1, 1, 1]
